Residential MCE Bill (Generation Charges Only)

"EV, Residential Rates for Electric Vehicle Owners" plus "Deep Green (Residential)"

In [ ]:
import os, sys
PWD = os.getenv('PWD')
os.chdir(PWD)
sys.path.insert(0, os.getenv('PWD'))
os.environ.setdefault("DJANGO_SETTINGS_MODULE", "local_settings.py")
import django
django.setup()

In [ ]:
from cost.utility_rate.models import RatePlan
from load.customer.models import Meter

In [ ]:
from datetime import datetime

# Step 1. Choose start and end_limit dates (Jan. 1 2018 through Jan. 31 2018)
start = datetime(2018, 1, 1)
end_limit = datetime(2018, 2, 1)

# Step 2. Get meter data
meter = Meter.objects.get(sa_id=8943913372)
intervalframe = meter.intervalframe.filter_by_datetime(start=start, end_limit=end_limit)

# Step 3. Get rates (OpenEI format)
ev_plan = RatePlan.objects.get(name="EV, Residential Rates for Electric Vehicle Owners")
ev_rates = ev_plan.get_latest_rate_collection(start)
ev_openei_rate_data = ev_rates.openei_rate_data

deep_green_plan = RatePlan.objects.get(name="Deep Green (Residential)")
deep_green_rates = deep_green_plan.get_latest_rate_collection(start)
deep_green_openei_rate_data = deep_green_rates.openei_rate_data

In [ ]:
from beo_datastore.libs.bill import ValidationBill

# Step 4. Generate bills
ev_bill = ValidationBill(intervalframe=intervalframe, openei_rate_data=ev_openei_rate_data)
deep_green_bill = ValidationBill(intervalframe=intervalframe, openei_rate_data=deep_green_openei_rate_data)

ev_bill.dataframe
# ev_bill.total
# deep_green_bill
# deep_green_bill.total

In [ ]:
from beo_datastore.libs.bill import BillingCollection

# Step 5. Consolodate bills
bill = BillingCollection(bills=[ev_bill, deep_green_bill])

bill.dataframe
# bill.total